In [1]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Double Peaked SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 13 January 2022
# Last Revision: 28 June 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [5]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [4]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
    tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
    #
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
    #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
    return tunique, tmatches, idx1, idx2



In [15]:
# Here we're loading in the double-peaked emission line galaxy catalog of Comerford2013
comerford2013 = ((Table.read('Tables/Comerford2013/table1.dat', readme = 'Tables/Comerford2013/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---','HbVoff','e_HbVoff','O3Voff','e_O3Voff','HaVoff','e_HaVoff','O3/Hb','e_O3/Hb','N2/Ha','e_N2/Ha','S2/Ha','e_S2/Ha','O1/Ha','e_O1/Ha','Class','n_Class'])
# table1.dat is a modified version of Wang's catalog in which I've added a duplicate row for each target
# since all of these are candidate dual AGN systems

# Comerford2013 looked at 173 Type 2 AGNs in Deep2 and found only two double-peaked dual AGN candidates \
# and found five offset AGNs. We are retaining only the double peaked duals.s

comerford2013 = comerford2013[(comerford2013['NDWFS'] == 'J143208.27+353255.5') | (comerford2013['NDWFS'] == 'J143359.71+351020.5')]

comerford2013['Coordinates'] = comerford2013['NDWFS'].str.slice(start=1)
comerford2013['RA_test'] = comerford2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
comerford2013['Dec_test'] = comerford2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
comerford2013['RA'] = comerford2013['RA_test'].str.slice(start=0, stop=2)+":"+comerford2013['RA_test'].str.slice(start=2, stop=4)+":"+comerford2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
comerford2013['Dec'] = comerford2013['Dec_test'].str.slice(start=0, stop=3)+":"+comerford2013['Dec_test'].str.slice(start=3, stop=5)+":"+comerford2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
comerford2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)


comerford2013['Name'] = comerford2013['NDWFS']
comerford2013['Name2'] = comerford2013['NDWFS']
comerford2013['z1'] = comerford2013['z']
comerford2013['z2'] = comerford2013['z']
comerford2013['z1_type'] = "spec"
comerford2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(comerford2013,comerford2013['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2013['RA'], dec = comerford2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2013['RA1_deg'] = coordconvert.ra.degree
comerford2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2013['RA2'] = comerford2013['RA']
comerford2013['Dec2'] = comerford2013['Dec']

comerford2013['RA2_deg'] = comerford2013['RA1_deg']
comerford2013['Dec2_deg'] = comerford2013['Dec1_deg']

# Adding details about the coordinates
comerford2013['Equinox'] = "J2000"
comerford2013['Coordinate_waveband'] = "Optical"
comerford2013['Coordinate_Source'] = "AGES"

comerford2013['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2013['Brightness1'] = "-99"
comerford2013['Brightness_band1'] = "-99"
comerford2013['Brightness_type1'] = "-99"

comerford2013['Brightness2'] = "-99"
comerford2013['Brightness_band2'] = "-99"
comerford2013['Brightness_type2'] = "-99"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
comerford2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#*******
#******************the cosmo arcsec to kpc command needs to be fixed! It is deprecated apparently!
#comerford2013['Sep(kpc)'] = comerford2013['Sep']*((cosmo.arcsec_per_kpc_proper(comerford2013['z']))**(-1))
#************
#*******


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
comerford2013['delta_z'] = comerford2013['z1']-comerford2013['z2']
comerford2013['dV'] = (2.99e+5)*((1+comerford2013['z1'])**2 - (1+comerford2013['z2'])**2)/((1+comerford2013['z1'])**2+(1+comerford2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
comerford2013['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
comerford2013['Confirmation Method'] = "-99"
comerford2013['Paper(s)'] = "Comerford+2013"
comerford2013['BibCode(s)'] = "2013ApJ...777...64C"
comerford2013['DOI(s)'] = "https://doi:10.1088/0004-637X/777/1/64"

## And dropping any columns that we don't need....
#comerford2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)
#
## Rearranging the columns and renaming columns now...
#comerford2013 = comerford2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#comerford2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


#comerford2013

In [7]:
# Loading in the tables from Comerford+2014 here now...

comerford2014 = ((Table.read('Tables/Comerford2014/table1.dat', readme = 'Tables/Comerford2014/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])

comerford2014['Name1'] = comerford2014['SDSS']
comerford2014['Name2'] = comerford2014['SDSS']
comerford2014['z2'] = comerford2014['z']
comerford2014['z1_type'] = "spec"
comerford2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(comerford2014,comerford2014['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = comerford2014['RA'], dec = comerford2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
comerford2014['RA1_deg'] = coordconvert.ra.degree
comerford2014['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
comerford2014['RA2'] = comerford2014['RA']
comerford2014['Dec2'] = comerford2014['Dec']

comerford2014['RA2_deg'] = comerford2014['RA1_deg']
comerford2014['Dec2_deg'] = comerford2014['Dec1_deg']

# Adding details about the coordinates
comerford2014['Equinox'] = "J2000"
comerford2014['Coordinate_waveband'] = "Optical"
comerford2014['Coordinate_Source'] = "SDSS"

comerford2014['System Type'] = 'Offset AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
comerford2014['Brightness1'] = -100
comerford2014['Brightness_band1'] = -100
comerford2014['Brightness_type1'] = -100

comerford2014['Brightness2'] = -100
comerford2014['Brightness_band2'] = -100
comerford2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
comerford2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#comerford2014['Sep(kpc)'] = comerford2014['Sep']*((cosmo.arcsec_per_kpc_proper(comerford2014['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#comerford2014['delta_z'] = comerford2014['z']-comerford2014['z2']
comerford2014['dV'] = (2.99e+5)*((1+comerford2014['z'])**2 - (1+comerford2014['z2'])**2)/((1+comerford2014['z'])**2+(1+comerford2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
comerford2014['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
comerford2014['Confirmation Method'] = "-99"
comerford2014['Paper(s)'] = "Comerford+2014"
comerford2014['BibCode(s)'] = "2014ApJ...789..112C"
comerford2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/789/2/112"

comerford2014['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#comerford2014.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#comerford2014 = comerford2014[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#comerford2014.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

#comerford2014


comerford2014




,SDSS,z,e_z,DelVB,e_DelVB,DelVF,e_DelVF,DelVW,e_DelVW,Name1,Name2,z2,z1_type,z2_type,RA,Dec,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J001828.09-003412.3,0.069291,0.000017,-64.4,14.7,-70.0,14.7,-67.2,10.4,J001828.09-003412.3,J001828.09-003412.3,0.069291,spec,spec,00:18:28.09,-00:34:12.3,4.617042,-0.570083,00:18:28.09,-00:34:12.3,4.617042,-0.570083,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
1,J002312.35+003956.2,0.072648,0.000020,47.9,14.8,54.8,14.7,51.4,10.4,J002312.35+003956.2,J002312.35+003956.2,0.072648,spec,spec,00:23:12.35,+00:39:56.2,5.801458,0.665611,00:23:12.35,+00:39:56.2,5.801458,0.665611,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
2,J003908.37-105833.0,0.065406,0.000020,-72.6,14.8,-52.1,14.8,-62.4,10.5,J003908.37-105833.0,J003908.37-105833.0,0.065406,spec,spec,00:39:08.37,-10:58:33.0,9.784875,-10.975833,00:39:08.37,-10:58:33.0,9.784875,-10.975833,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
3,J003948.38-090834.5,0.037475,0.000023,52.6,14.7,49.7,14.7,51.1,10.4,J003948.38-090834.5,J003948.38-090834.5,0.037475,spec,spec,00:39:48.38,-09:08:34.5,9.951583,-9.142917,00:39:48.38,-09:08:34.5,9.951583,-9.142917,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
4,J004000.73-005146.0,0.111223,0.000017,57.6,18.0,79.7,16.6,69.6,12.2,J004000.73-005146.0,J004000.73-005146.0,0.111223,spec,spec,00:40:00.73,-00:51:46.0,10.003042,-0.862778,00:40:00.73,-00:51:46.0,10.003042,-0.862778,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
5,J004408.00-005943.8,0.130096,0.000017,-80.0,19.0,-83.9,18.4,-82.0,13.2,J004408.00-005943.8,J004408.00-005943.8,0.130096,spec,spec,00:44:08.00,-00:59:43.8,11.033333,-0.995500,00:44:08.00,-00:59:43.8,11.033333,-0.995500,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
6,J011031.30+003636.5,0.122260,0.000012,80.1,17.7,74.0,17.5,77.0,12.5,J011031.30+003636.5,J011031.30+003636.5,0.122260,spec,spec,01:10:31.30,+00:36:36.5,17.630417,0.610139,01:10:31.30,+00:36:36.5,17.630417,0.610139,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
7,J011333.06+002947.9,0.044105,0.000016,-65.6,15.6,-81.9,15.0,-74.0,10.8,J011333.06+002947.9,J011333.06+002947.9,0.044105,spec,spec,01:13:33.06,+00:29:47.9,18.387750,0.496639,01:13:33.06,+00:29:47.9,18.387750,0.496639,J2000,Optical,SDSS,Offset AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Comerford+2014,2014ApJ...789..112C,https://doi.org/10.1088/0004-637X/789/2/112,
8,J013258.91-102707.0,0.032219,0.000016,54.3,14.6,58.2,14.6,56.2,10.3,J013258.91-102707.0,J013258.91-102707.0,0.032219,spec,spec,01:32:58.91,-10:27:07.0,23.245458,-10.451944,01:32:58.91,-10:27:07.0,23.245458,-10.451944,J20

In [ ]:
# Adding in information from Allen+2015



allenobjs = ['J120401.97+012641.6','J011333.06+002947.9'] # Allen+ used these: J120401.97+012641.5 J011333.05+002948.0

# Adding the DOI, author, and bibcode info to all of the Comerford+2014 rows here in the matches table...
for index, row in comerford2014.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Allen+2015'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2015MNRAS.451.2780A' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/stv1121'

drobjs = ['J120401.97+012641.5']
for index, row in comerford2014.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Notes']+=' Allen+ use IFS observations to show that the offset emission line(s) used to flag this target originally is not the result of a binary system but due to large-scale motions of gas in the galaxy. If this system does constitute a merger they cannot rule out the possible presence of two SMBHs. There is a suggestion that it could be a merger but no firm evidence. '

drobjs = ['J011333.05+002948.0']
for index, row in comerford2014.iterrows():
    if row['Name1'] in allenobjs:
        the_whills.at[index, 'Notes']+=' Allen+ uses IFS observations to show that the offset emission line(s) are likely due to an outflow rather than the motion of a NLR. They argue strongly against a binary scenario.'



In [ ]:
# Adding in the information from Barrows+

barrowsobjs = ['J111519.97+542316.6'] #J111519.98+542316.65

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in liu2011.iterrows():
    if row['Name'] in drobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Barrows+2016'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
        #the_whills.at[index, 'System Type']='Dual AGN'
        the_whills.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
        #the_whills.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'
    


In [ ]:
# Adding in information from Muller-Sanchez+2015

msobjs = ['J101847.57+294114.10','J105553.64+152027.40','J111729.21+614015.30','J134640.79+522836.60']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in comerford2014.iterrows():
    if row['Name'] in msobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Müller-Sánchez'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2016ApJ...830...50M' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/830/1/50'
        #the_whills.at[index, 'Notes']+='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'


drobjs = ['J101847.57+294114.10','J105553.64+152027.40','J134640.79+522836.60']
for index, row in comerford2014.iterrows():
    if row['Name'] in msobjs:
        the_whills.at[index, 'Notes']+=' Muller-Sanchez+2016 used OSIRIS IFS observations to show that the observed spatial distribution and kinematics of the ionized gas indicate the presence of outflows and are spatially offset from the center of the galaxy. The spatially offset peaks in the emission can be explained via shocks. The integrated OSIRIS spectrum shows the same velocity offset as observed in SDSS but the spectral profiles are different; the near-IR lines show clear asymmetric wings not seen in the optical.'

drobjs = ['J111729.21+614015.30']
for index, row in comerford2014.iterrows():
    if row['Name'] in msobjs:
        the_whills.at[index, 'Notes']+=' Muller-Sanchez+2016 used OSIRIS IFS observations to show a counter-rotating circumnuclear disk contains the peak of the offset Pa-alpha emission. The most plausible origin of this enhanced emission is that it lies at an intersection zone between the nuclear disk and bar of the galaxy where cloud-cloud collisions drive the emission. The integrated OSIRIS spectrum shows the same velocity offset as observed in SDSS.'




